## **Informações**

Código para produção de dashboard de casos e vacinação referente à pandemia do Covid-19

**Dashboard:**
Google Data Studio (link).

**Processamento:**
Kaggle Notebook (link).

**GitHub:**


**Fontes:**
Casos pela universidade John Hopkins: [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports)

Vacinação pela universidade de Oxford:  [link](https://covid.ourworldindata.org/data/owid-covid-data.csv)



## **1. Dados de Vacinação**

### **1.1. Extração dos dados**

In [1]:
import pandas as pd

In [2]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-2-608e6d2dd56d>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [3]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772,NaN,NaN,NaN,NaN


### **1.2. Entendendo os dados**

In [4]:
vaccines.shape

(429435, 67)

In [5]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429435 entries, 0 to 429434
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   iso_code                                    429435 non-null  object        
 1   continent                                   402910 non-null  object        
 2   location                                    429435 non-null  object        
 3   date                                        429435 non-null  datetime64[ns]
 4   total_cases                                 411804 non-null  float64       
 5   new_cases                                   410159 non-null  float64       
 6   new_cases_smoothed                          408929 non-null  float64       
 7   total_deaths                                411804 non-null  float64       
 8   new_deaths                                  410608 non-null  float64      

#### 1.2.1. Limpeza dos dados

Organizando chaves temporais

In [6]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))
vaccines['day'] = vaccines['date'].dt.day

Renomeando colunas

In [7]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

#### 1.2.2. Filtrando informações

In [8]:
#Filtrando o período
vaccines = vaccines.query('month >= "2020-03" & month < "2023-01"')

In [9]:
#Filtrando países do continente norte americano

north_america_countries = [
   "Canada",  "Mexico", "United States", "US"
]


vaccines = vaccines[vaccines['country'].isin(north_america_countries)]

In [10]:
vaccines['country'].unique()

array(['Canada', 'Mexico', 'United States'], dtype=object)

Preenchendo valores nulos

In [11]:
vaccines_tratado_ = None
vaccines_is_empty = None
for country in north_america_countries:
    vaccines_tratado = vaccines[vaccines['country'].isin([country])].fillna(method='ffill')
    if vaccines_is_empty:
        vaccines_tratado_ = vaccines_tratado
        vaccines_is_empty = False
    else:
        vaccines_tratado_ = pd.concat([vaccines_tratado_, vaccines_tratado], axis=0, ignore_index=True)

vaccines_tratado = vaccines_tratado_
vaccines_tratado_ = None

<ipython-input-11-04de9203710e>:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vaccines_tratado = vaccines[vaccines['country'].isin([country])].fillna(method='ffill')


### 1.3. Calculando **métricas**

In [12]:
#Organizando data
vaccines_tratado['full date'] = pd.to_datetime(vaccines_tratado['month'] + '-' + vaccines_tratado["day"].astype(str))

In [13]:
vaccines_grouped = vaccines_tratado.groupby(by=["full date", "country"]).agg({
    'population': 'sum',
    'total': 'sum',
    'one_shot': 'sum',
    'two_shots': 'sum',
    'three_shots':'sum',
}).reset_index()

In [14]:
vaccines_grouped

,full date,country,population,total,one_shot,two_shots,three_shots
0,2020-03-01,Canada,38454328,0.0,0.0,0.0,0.0
1,2020-03-01,Mexico,127504120,0.0,0.0,0.0,0.0
2,2020-03-01,United States,338289856,0.0,0.0,0.0,0.0
3,2020-03-02,Canada,38454328,0.0,0.0,0.0,0.0
4,2020-03-02,Mexico,127504120,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3103,2022-12-30,Mexico,127504120,223158993.0,97179493.0,81849962.0,57026630.0
3104,2022-12-30,United States,338289856,666888577.0,268531382.0,229575525.0,133062763.0
3105,2022-12-31,Canada,38454328,96303716.0,34618256.0,31731097.0,29954363.0
3106,2022-12-31,Mexico,127504120,223158993.0,97179493.0,81849962.0,57026630.0


In [15]:
day = list(vaccines_grouped['full date'].unique())

for period in day:
  period_data = vaccines_grouped[vaccines_grouped['full date']==period]
  vaccines_grouped.loc[vaccines_grouped['full date'] == period, 'one_shot_perc'] = round(period_data['one_shot'] / period_data['population'], 4)
  vaccines_grouped.loc[vaccines_grouped['full date'] == period, 'two_shots_perc'] = round(period_data['two_shots'] / period_data['population'], 4)
  vaccines_grouped.loc[vaccines_grouped['full date'] == period, 'three_shots_perc'] = round(period_data['three_shots'] / period_data['population'], 4)

In [16]:
vaccines_grouped = vaccines_grouped[['full date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc']]

In [17]:
vaccines_grouped.shape

(3108, 10)

In [18]:
vaccines_grouped.tail()

,full date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc
3103,2022-12-30,Mexico,127504120,223158993.0,97179493.0,0.7622,81849962.0,0.6419,57026630.0,0.4473
3104,2022-12-30,United States,338289856,666888577.0,268531382.0,0.7938,229575525.0,0.6786,133062763.0,0.3933
3105,2022-12-31,Canada,38454328,96303716.0,34618256.0,0.9002,31731097.0,0.8252,29954363.0,0.7790
3106,2022-12-31,Mexico,127504120,223158993.0,97179493.0,0.7622,81849962.0,0.6419,57026630.0,0.4473
3107,2022-12-31,United States,338289856,666952241.0,268541239.0,0.7938,229580638.0,0.6787,133062763.0,0.3933


### 1.4. Carregamento dos dados

In [19]:
# Alterar nome 'United States' para 'US' no DataFrame vaccines_grouped
states_map = {
    'United States': 'US'
}

vaccines_grouped['country'] = vaccines_grouped['country'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [20]:
vaccines_grouped = vaccines_grouped.rename(columns ={'country':'Country_Region'})

In [21]:
vaccines_grouped.tail()

,full date,Country_Region,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc
3103,2022-12-30,Mexico,127504120,223158993.0,97179493.0,0.7622,81849962.0,0.6419,57026630.0,0.4473
3104,2022-12-30,US,338289856,666888577.0,268531382.0,0.7938,229575525.0,0.6786,133062763.0,0.3933
3105,2022-12-31,Canada,38454328,96303716.0,34618256.0,0.9002,31731097.0,0.8252,29954363.0,0.7790
3106,2022-12-31,Mexico,127504120,223158993.0,97179493.0,0.7622,81849962.0,0.6419,57026630.0,0.4473
3107,2022-12-31,US,338289856,666952241.0,268541239.0,0.7938,229580638.0,0.6787,133062763.0,0.3933


In [22]:
vaccines_grouped.to_csv('./covid-vaccines.csv', sep=',', index=False)

##**2. Dados da Covid**

### **2.1. Extração dos dados**

In [23]:
import pandas as pd
import os

In [24]:
#Clonando repositório
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 682302, done.
remote: Total 682302 (delta 0), reused 0 (delta 0), pack-reused 682302 (from 1)
Receiving objects: 100% (682302/682302), 7.85 GiB | 24.75 MiB/s, done.
Resolving deltas: 100% (598414/598414), done.
Updating files: 100% (2434/2434), done.


In [25]:
# Caminho para o diretório onde estão os arquivos CSV
path = "/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"

In [26]:
# Identificar todas as colunas presentes em todos os arquivos CSV
all_columns = set()
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filepath = os.path.join(path, filename)
        df = pd.read_csv(filepath)
        all_columns.update(df.columns)

In [27]:
dfs = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filepath = os.path.join(path, filename)
        df = pd.read_csv(filepath)
        df = df.reindex(columns=all_columns)
        df.reset_index(drop=True, inplace=True)
        dfs.append(df)

In [28]:
cases = pd.concat(dfs, ignore_index=True)

### **2.2. Entendendo os dados**

In [29]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4287473 entries, 0 to 4287472
Data columns (total 21 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Case-Fatality_Ratio  float64
 1   Country/Region       object 
 2   Country_Region       object 
 3   Long_                float64
 4   Incident_Rate        float64
 5   Admin2               object 
 6   Incidence_Rate       float64
 7   Latitude             float64
 8   Deaths               float64
 9   Lat                  float64
 10  Recovered            float64
 11  Confirmed            float64
 12  Case_Fatality_Ratio  float64
 13  Last Update          object 
 14  Last_Update          object 
 15  Province/State       object 
 16  FIPS                 float64
 17  Province_State       object 
 18  Active               float64
 19  Longitude            float64
 20  Combined_Key         object 
dtypes: float64(13), object(8)
memory usage: 686.9+ MB


In [30]:
cases.isnull().sum()

,0
Case-Fatality_Ratio,3653921
Country/Region,4277676
Country_Region,9797
Long_,104094
Incident_Rate,948818
Admin2,782109
Incidence_Rate,3657406
Latitude,4281981
Deaths,433
Lat,104094


#### 2.2.1. Limpeza do dataframe

In [31]:
#Dropar valores nulos
cases = cases.dropna(subset=['Country_Region']).reset_index(drop=True)

In [32]:
cases.head()

,Case-Fatality_Ratio,Country/Region,Country_Region,Long_,Incident_Rate,Admin2,Incidence_Rate,Latitude,Deaths,Lat,...,Confirmed,Case_Fatality_Ratio,Last Update,Last_Update,Province/State,FIPS,Province_State,Active,Longitude,Combined_Key
0,0.000000,NaN,US,-82.461707,NaN,Abbeville,187.548416,NaN,0.0,34.223334,...,46.0,NaN,NaN,2020-06-04 02:33:14,NaN,45001.0,South Carolina,46.0,NaN,"Abbeville, South Carolina, US"
1,5.829596,NaN,US,-92.414197,NaN,Acadia,718.833105,NaN,26.0,30.295065,...,446.0,NaN,NaN,2020-06-04 02:33:14,NaN,22001.0,Louisiana,420.0,NaN,"Acadia, Louisiana, US"
2,1.259182,NaN,US,-75.632346,NaN,Accomack,2949.003590,NaN,12.0,37.767072,...,953.0,NaN,NaN,2020-06-04 02:33:14,NaN,51001.0,Virginia,941.0,NaN,"Accomack, Virginia, US"
3,2.689487,NaN,US,-116.241552,NaN,Ada,169.855083,NaN,22.0,43.452658,...,818.0,NaN,NaN,2020-06-04 02:33:14,NaN,16001.0,Idaho,796.0,NaN,"Ada, Idaho, US"
4,0.000000,NaN,US,-94.471059,NaN,Adair,125.838926,NaN,0.0,41.330756,...,9.0,NaN,NaN,2020-06-04 02:33:14,NaN,19001.0,Iowa,9.0,NaN,"Adair, Iowa, US"


In [33]:
cases = cases.drop(columns=['FIPS', 'Admin2', 'Incident_Rate', 'Case-Fatality_Ratio', 'Longitude', 'Latitude',
                            'Last Update', 'Active','Case_Fatality_Ratio', 'Combined_Key','Incident_Rate',
                            'Incidence_Rate','Province/State','Country/Region','Province_State'], axis=1)

In [34]:
#cases.fillna(value=0, inplace=True)

In [35]:
cases.isnull().sum()

,0
Country_Region,0
Long_,94297
Deaths,1
Lat,94297
Recovered,2828055
Confirmed,0
Last_Update,0


In [36]:
#Organizando colunas
cases = cases[["Last_Update",'Country_Region', "Lat", "Long_", "Confirmed",
               "Recovered","Deaths"]]

#### 2.2.2. Organizando a série temporal

In [37]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [38]:
cases.head()

,Last_Update,Country_Region,Lat,Long_,Confirmed,Recovered,Deaths
0,2020-06-04 02:33:14,US,34.223334,-82.461707,46.0,0.0,0.0
1,2020-06-04 02:33:14,US,30.295065,-92.414197,446.0,0.0,26.0
2,2020-06-04 02:33:14,US,37.767072,-75.632346,953.0,0.0,12.0
3,2020-06-04 02:33:14,US,43.452658,-116.241552,818.0,0.0,22.0
4,2020-06-04 02:33:14,US,41.330756,-94.471059,9.0,0.0,0.0


In [39]:
# Converter a coluna 'Last_Update' para datetime
cases['Last_Update'] = pd.to_datetime(cases['Last_Update'], errors='coerce')

# Extrair apenas a data no formato aaaa-mm-dd
cases['Date_Only'] = cases['Last_Update'].dt.strftime('%Y-%m-%d')

In [40]:
#Inserindo ano, mês e dia
cases['Year'] = cases['Last_Update'].dt.year
cases['Month'] = cases['Last_Update'].dt.month
cases['Day'] = cases['Last_Update'].dt.day

In [41]:
cases['Date_Only'] = pd.to_datetime(cases['Date_Only'])
cases['month_year'] = cases['Date_Only'].dt.strftime('%Y-%m')

### 2.3. Filtrando valores

In [42]:
cases = cases[cases['Country_Region'].isin(north_america_countries)]
cases = cases.query('month_year >= "2020-03" & month_year < "2023-01"')

In [43]:
cases['month_year'].unique()

array(['2020-06', '2021-04', '2021-11', '2020-12', '2020-08', '2021-01',
       '2021-10', '2021-07', '2021-12', '2021-09', '2021-05', '2022-07',
       '2022-06', '2020-04', '2022-03', '2022-09', '2022-08', '2020-11',
       '2022-01', '2020-07', '2022-10', '2022-11', '2020-10', '2020-09',
       '2020-03', '2022-12', '2020-05', '2022-04', '2021-02', '2022-05',
       '2021-08', '2021-06', '2021-03', '2022-02'], dtype=object)

In [44]:
cases = cases.drop(columns=['Last_Update', 'Lat', "Long_"], axis=1)

In [45]:
cases['Date_Only'] = pd.to_datetime(cases['Date_Only'])

In [46]:
cases_grouped = cases.groupby(by=["Date_Only", "Country_Region"]).agg({
    'Confirmed': 'sum',
    'Recovered': 'sum',
    'Deaths': 'sum',
}).reset_index()

In [47]:
cases_grouped = cases_grouped.rename(columns={"Date_Only":"full date"})

### 2.4. Calculando Métricas


In [48]:
cases_grouped

,full date,Country_Region,Confirmed,Recovered,Deaths
0,2020-03-23,Canada,3297.0,0.0,29.0
1,2020-03-23,Mexico,316.0,4.0,3.0
2,2020-03-23,US,43719.0,0.0,580.0
3,2020-03-24,Canada,3996.0,110.0,34.0
4,2020-03-24,Mexico,367.0,4.0,4.0
...,...,...,...,...,...
3007,2022-12-30,Mexico,7234467.0,0.0,331099.0
3008,2022-12-30,US,100722232.0,0.0,1092519.0
3009,2022-12-31,Canada,4504318.0,0.0,49228.0
3010,2022-12-31,Mexico,7234467.0,0.0,331099.0


In [49]:
cases_grouped['Case_Fatality_Ratio'] = cases_grouped['Deaths'] / cases_grouped['Confirmed']
cases_grouped['Active'] = cases_grouped["Confirmed"] - cases_grouped["Recovered"] - cases_grouped["Deaths"]
cases_grouped['Incident_Rate'] = cases_grouped['Confirmed'] / 100000.0

In [50]:
# Ordenar o DataFrame por país e data
cases_grouped = cases_grouped.sort_values(by=['Country_Region', 'full date'])

# Calcular a diferença diária
cases_grouped['confirmed_1d'] = cases_grouped.groupby('Country_Region')['Confirmed'].diff(periods=1)
cases_grouped['deaths_1d'] = cases_grouped.groupby('Country_Region')['Deaths'].diff(periods=1)

# Calcular a média móvel de 7 dias
cases_grouped['confirmed_moving_avg_7d'] = cases_grouped.groupby('Country_Region')['confirmed_1d'].rolling(window=7).mean().reset_index(level=0, drop=True)
cases_grouped['deaths_moving_avg_7d'] = cases_grouped.groupby('Country_Region')['deaths_1d'].rolling(window=7).mean().reset_index(level=0, drop=True)

# Calcular a taxa de variação de 14 dias (se desejado)
cases_grouped['confirmed_moving_avg_7d_rate_14d'] = cases_grouped.groupby('Country_Region')['confirmed_moving_avg_7d'].pct_change(periods=14)
cases_grouped['deaths_moving_avg_7d_rate_14d'] = cases_grouped.groupby('Country_Region')['deaths_moving_avg_7d'].pct_change(periods=14)


<ipython-input-50-e3a47416e5de>:13: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  cases_grouped['confirmed_moving_avg_7d_rate_14d'] = cases_grouped.groupby('Country_Region')['confirmed_moving_avg_7d'].pct_change(periods=14)
<ipython-input-50-e3a47416e5de>:14: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  cases_grouped['deaths_moving_avg_7d_rate_14d'] = cases_grouped.groupby('Country_Region')['deaths_moving_avg_7d'].pct_change(periods=14)


### 2.4. Carregamento dos dados

In [51]:
cases_grouped.to_csv('./covid-cases.csv', sep=',', index=False)

In [52]:
cases_grouped.tail()

,full date,Country_Region,Confirmed,Recovered,Deaths,Case_Fatality_Ratio,Active,Incident_Rate,confirmed_1d,deaths_1d,confirmed_moving_avg_7d,deaths_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,deaths_moving_avg_7d_rate_14d
2999,2022-12-27,US,100393963.0,0.0,1090249.0,0.010860,99303714.0,1003.93963,12432.0,29.0,60248.285714,356.714286,2.227406,-4.982456
3002,2022-12-28,US,100504898.0,0.0,1090605.0,0.010851,99414293.0,1005.04898,110935.0,356.0,64385.000000,323.857143,0.004905,-0.234897
3005,2022-12-29,US,100618242.0,0.0,1091595.0,0.010849,99526647.0,1006.18242,113344.0,990.0,54553.000000,316.428571,-0.154674,-0.220345
3008,2022-12-30,US,100722232.0,0.0,1092519.0,0.010847,99629713.0,1007.22232,103990.0,924.0,55667.571429,363.285714,-0.162159,-0.081949
3011,2022-12-31,US,100756710.0,0.0,1092735.0,0.010845,99663975.0,1007.56710,34478.0,216.0,54987.857143,364.571429,-0.203961,-0.148198


In [53]:
vaccines_grouped.tail()

,full date,Country_Region,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc
3103,2022-12-30,Mexico,127504120,223158993.0,97179493.0,0.7622,81849962.0,0.6419,57026630.0,0.4473
3104,2022-12-30,US,338289856,666888577.0,268531382.0,0.7938,229575525.0,0.6786,133062763.0,0.3933
3105,2022-12-31,Canada,38454328,96303716.0,34618256.0,0.9002,31731097.0,0.8252,29954363.0,0.7790
3106,2022-12-31,Mexico,127504120,223158993.0,97179493.0,0.7622,81849962.0,0.6419,57026630.0,0.4473
3107,2022-12-31,US,338289856,666952241.0,268541239.0,0.7938,229580638.0,0.6787,133062763.0,0.3933
